In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import timeit
import unittest

## Please DONOT remove these lines. 
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [ ]:
device = torch.device("cuda")
print(device)

In [ ]:
import numpy

def compute_mean_std(svhn_dataset):
    """compute the mean and std of svhn dataset
    Args:
        svhn_training_dataset or cifar100_test_dataset
        witch derived from class torch.utils.data
    
    Returns:
        a tuple contains mean, std value of entire dataset
    """

    data_r = numpy.dstack([svhn_dataset[i][0][0, :, :] for i in range(len(svhn_dataset))])
    data_g = numpy.dstack([svhn_dataset[i][0][1, :, :] for i in range(len(svhn_dataset))])
    data_b = numpy.dstack([svhn_dataset[i][0][2, :, :] for i in range(len(svhn_dataset))])
    mean = numpy.mean(data_r), numpy.mean(data_g), numpy.mean(data_b)
    std = numpy.std(data_r), numpy.std(data_g), numpy.std(data_b)

    return mean, std

# mean, std = compute_mean_std(svhn_dataset)
# print(mean)
# print(std)

In [ ]:
#import torchvision
import scipy
"""
    Applying various transformations/preprocessing techniques on the dataset 
"""

transform_svhn_train = transforms.Compose([
        transforms.Resize(64, interpolation = 2), #increasing the image size using bilinear interpolation
        #transforms.RandomResizedCrop(64),
        #transforms.GaussianBlur(kernel_size = 3, sigma=(0.1, 2.0)),
        #torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
        transforms.ToTensor(), # convert the image to a pytorch tensor
        #transforms.RandomHorizontalFlip(), #random horizontal flipping with a probability of 0.5
        #transforms.ColorJitter(brightness=0.4, saturation=0.4), # random color jittering with saturation and brightness
        #transforms.Normalize((0.437, 0.443, 0.472), (0.198, 0.201, 0.197)),
        transforms.Normalize((0.438, 0.444, 0.473), (0.195, 0.198, 0.195)),
        
        ])

transform_svhn_test = transforms.Compose([
        transforms.Resize(64, interpolation = 2), #increasing the image size using bilinear interpolation
        transforms.ToTensor(), # convert the image to a pytorch tensor
        transforms.Normalize((0.438, 0.444, 0.473), (0.195, 0.198, 0.195)),
        
        ])


In [ ]:
import torchvision

train_dataset = torchvision.datasets.SVHN(root = "./data_svhn", split = "train", download = True, transform = transform_svhn_train)
test_dataset = torchvision.datasets.SVHN(root = "./data_svhn", split = "test", download = True, transform = transform_svhn_test)

# print(train_dataset[0][0])
# image = train_dataset[0][0].permute(1,2,0)
# image -= image.min() 
# image /= image.max()
# print(image)
# image = 255*image
# plt.imshow(image)
# print(image)
# mean, std = compute_mean_std(train_dataset)
# print(mean)
# print(std)

In [ ]:
"""
    Creating dataloaders for train and test datasets
        Args:
        batch_size = 32
        shuffle = True for training set
"""

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = True)

print(train_dataloader)

In [ ]:
# Function to train the model

def svhn_train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_id, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        model.zero_grad() #calling the zero_grad function to flush out the gradients 
        output = model(data)
        loss = nn.NLLLoss()(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_id%1000 == 0:
            print('Train epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_id*len(data), 
                        len(train_loader.dataset), 100. * batch_id/len(train_loader), loss.item()))

In [ ]:
def svhn_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            target = torch.tensor(target, dtype = torch.long, device = device)
            #print(target)
            output = model(data)
            test_loss += nn.NLLLoss()(output, target).item() #sum up batch loss
            pred = output.argmax(dim =1, keepdim=True) #this is the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
    
    return test_loss


In [ ]:
class SVHNNet(nn.Module):
    def __init__(self):
        super(SVHNNet, self).__init__()
        #### YOUR CODE STARTS HERE ####
        self.conv1 = nn.Conv2d(3, 16, 3, stride=1)
        self.norm1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, stride=1)
        self.norm2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, stride=1)
        self.norm3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, stride=1)
        self.norm4 = nn.BatchNorm2d(128)
        self.maxpool = nn.MaxPool2d(2, stride=2)
        self.conv5 = nn.Conv2d(128, 256, 3, stride=1)
        self.norm5 = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 256, 3, stride=1)
        self.norm6 = nn.BatchNorm2d(256)
        self.conv7 = nn.Conv2d(256, 256, 3, stride=1)
        self.norm7 = nn.BatchNorm2d(256)
        self.conv8 = nn.Conv2d(256, 256, 3, stride=1)
        self.norm8 = nn.BatchNorm2d(256)
        self.dropout1 = nn.Dropout(p=0.25)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(in_features=256*10*10, out_features=1000)
        self.fc2 = nn.Linear(1000,400)
        self.fc3 = nn.Linear(400,100)
        self.fc4 = nn.Linear(100,10)
        self.act = nn.ReLU()
        #### YOUR CODE ENDS HERE ####

    def forward(self, x):
        # Use the layers defined above in a sequential way (folow the same as the layer definitions above) and 
        # write the forward pass, after each of conv1, conv2, conv3 and fc1 use a relu activation. 
        #### YOUR CODE STARTS HERE ####
        x = self.conv1(x)
        x = self.act(self.norm1(x))
        x = self.conv2(x)
        x = self.act(self.norm2(x))
        x = self.conv3(x)
        x = self.act(self.norm3(x))
        x = self.conv4(x)
        x = self.act(self.norm4(x))
        x = self.maxpool(x)
        x = self.conv5(x)
        x = self.act(self.norm5(x))
        x = self.conv6(x)
        x = self.act(self.norm6(x))
        x = self.conv7(x)
        x = self.act(self.norm7(x))
        x = self.conv8(x)
        x = self.act(self.norm8(x))
        x = self.maxpool(x)
        #x = self.dropout1(x)
        #x = self.dropout2(x)
        x = x.reshape(x.size(0), -1)
        #print(x.size())
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        x = self.act(self.fc4(x))
        #### YOUR CODE ENDS HERE ####
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
#model = SVHNNet().to(device)
model = torchvision.models.resnet18(pretrained=True)

model.fc = nn.Linear(512, 10)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Dropout(0.25),
    nn.Linear(512, 10),
    nn.LogSoftmax()
)

model.load_state_dict(torch.load("model_reg.pth"))



# for param in model.parameters():
#     param.requires_grad = False
    

model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0000000001) #weight_decay = 0.0001
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience = 3, verbose=True)
filepath = "model_reg.pth"

start = timeit.default_timer()
for epoch in range(1, 11):
    svhn_train(model, device, train_dataloader, optimizer, epoch)
    test_loss = svhn_test(model, device, test_dataloader)
    scheduler.step(test_loss)

torch.save(model.state_dict(), filepath)
stop = timeit.default_timer()
print('Total time taken: {} seconds'.format(int(stop - start)) )